In [8]:
import os
import re
import csv
import sys
import ast 
import json
import time
import urllib
import random
import spotipy
import requests
import datetime
import configparser
import numpy as np
import pandas as pd
import urllib.parse
from pprint import pprint
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
# spotifyAPI存取權限
client_id = '1a23ed1ca359485ab81d524206dba0f6'
client_secret = 'af0e14fdb19a49dfb19ef0c1ea1d42ea'
username = 'qnwv65t11cplaz4dikhl4mjgi'
scope = 'user-read-private user-read-playback-state user-modify-playback-state'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
# https://github.com/plamere/spotipy/blob/master/examples/show_album.py
# https://dev.to/helloiamarra/how-to-play-spotify-songs-and-show-the-album-art-using-spotipy-library-and-python-5eki
uri = 'spotify:album:5hVsQ9MdhzoPAxDB8ohTCr'
album = sp.album(uri)
pprint(album)

{'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5Rl15oVamLq7FbSb0NNBNy'},
              'href': 'https://api.spotify.com/v1/artists/5Rl15oVamLq7FbSb0NNBNy',
              'id': '5Rl15oVamLq7FbSb0NNBNy',
              'name': '5 Seconds of Summer',
              'type': 'artist',
              'uri': 'spotify:artist:5Rl15oVamLq7FbSb0NNBNy'}],
 'available_markets': ['JP'],
 'copyrights': [{'text': '© 2014 One Mode Productions Limited, under exclusive '
                         'licence to Universal Music Operations Limited',
                 'type': 'C'},
                {'text': 'A Capitol Records UK release; ℗ 2014 One Mode '
                         'Productions Limited, under exclusive licence to '
                         'Universal Music Operations Limited',
                 'type': 'P'}],
 'external_ids': {'upc': '00602537875252'},
 'external_urls': {'spotify': 'https://open.spotify.com/album/5hVsQ9MdhzoPAxDB8ohTCr'},
 'genres': 

In [51]:
# 抓專輯的總歌曲數
num = album['tracks']['total']
num

17

In [70]:
# 專輯中每首歌曲資訊(修改artist和song的最後一個數字代表不同首)
L_artist_id = []
L_artist_name = []
L_artist_uri = []
L_song_id = []
L_song_name = []
L_song_track_number = []
L_song_uri = []

for i in range(album['tracks']['total']):
    artist = album['tracks']['items'][0]['artists'][0]
    song = album['tracks']['items'][i]
    
    artist_id = artist['id']
    artist_name = artist['name']
    artist_uri = artist['uri']
    song_id = song['id']
    song_name = song['name']
    song_track_number = song['track_number']
    song_uri = song['uri']
    
    L_artist_id.append(artist_id)
    L_artist_name.append(artist_name)
    L_artist_uri.append(artist_uri)
    L_song_id.append(song_id)
    L_song_name.append(song_name)
    L_song_track_number.append(song_track_number)
    L_song_uri.append(song_uri)

df_album_tracks = pd.DataFrame(list(zip(L_artist_id,L_artist_name,L_artist_uri,L_song_id,L_song_name,L_song_track_number,L_song_uri)),columns=['artist_id','artist_name','artist_uri','song_id','song_name','song_track_number','song_uri'])
df_album_tracks

,artist_id,artist_name,artist_uri,song_id,song_name,song_track_number,song_uri
0,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer,spotify:artist:5Rl15oVamLq7FbSb0NNBNy,50iYq7AOtwvZBbjcYWR48v,She Looks So Perfect,1,spotify:track:50iYq7AOtwvZBbjcYWR48v
1,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer,spotify:artist:5Rl15oVamLq7FbSb0NNBNy,5RpXdNM0PK9EYCsBHCd4tk,Don't Stop,2,spotify:track:5RpXdNM0PK9EYCsBHCd4tk
2,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer,spotify:artist:5Rl15oVamLq7FbSb0NNBNy,17qRQOod3Ujpvfo5DJXAFn,Good Girls,3,spotify:track:17qRQOod3Ujpvfo5DJXAFn
3,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer,spotify:artist:5Rl15oVamLq7FbSb0NNBNy,51IDdV5iWWxjsrBU18vxNO,Kiss Me Kiss Me,4,spotify:track:51IDdV5iWWxjsrBU18vxNO
4,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer,spotify:artist:5Rl15oVamLq7FbSb0NNBNy,67gJbK5kY61AV8vy4y54Yn,18,5,spotify:track:67gJbK5kY61AV8vy4y54Yn
5,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer,spotify:artist:5Rl15oVamLq7FbSb0NNBNy,0RBQbrd0LeZSgHxCf7Y0oA,Everything I Didn't Say,6,spotify:track:0RBQbrd0LeZSgHxCf7Y0oA
6,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer,spotify:artist:5Rl15oVamLq7FbSb0NNBNy,5Hmwizz0A3mOQqiA67LGuE,Beside You,7,spotify:track:5Hmwizz0A3mOQqiA67LGuE
7,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer,spotify:artist:5Rl15oVamLq7FbSb0NNBNy,6lNgDgV7LrlkKLx8qRhVWw,End Up Here,8,spotify:track:6lNgDgV7LrlkKLx8qRhVWw
8,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer,spotify:artist:5Rl15oVamLq7FbSb0NNBNy,6XwffqgxTitQM9roYOGnH3,Long Way Home,9,spotify:track:6XwffqgxTitQM9roYOGnH3
9,5Rl15oVamLq7FbSb0NNBNy,5 Seconds of Summer,spotify:artist:5Rl15oVamLq7FbSb0NNBNy,1fYZ2d294iginGnpqTLdxu,Heartbreak Girl,10,spotify:track:1fYZ2d294iginGnpqTLdxu
